In [2]:
# Install the library:
#!python setup.py install --user

In [3]:
from kdmkr import datasets
from kdmkr import distillation
from kdmkr import evaluation
from kdmkr import loss
from kdmkr import model

In [4]:
from creme import stats

In [5]:
import pickle 
import torch
import tqdm

In [6]:
device     = 'cuda'
hidden_dim = 1000

In [6]:
fb15k237 = datasets.FB15K237(
    batch_size=1024, 
    negative_sample_size=256, 
    shuffle=True, 
    seed=42
)

In [6]:
teacher = model.RotatE(
    hidden_dim=1000, 
    n_entity=fb15k237.n_entity, 
    n_relation=fb15k237.n_relation, 
    gamma=9
)
teacher = teacher.to(device)

In [ ]:
optimizer_teacher = torch.optim.Adam(filter(lambda p: p.requires_grad, teacher.parameters()), lr = 0.00005)

max_step = 40000

bar = tqdm.tqdm(range(1, max_step), position=0)

metric = stats.RollingMean(1000)

evaluation = evaluation.Evaluation()

teacher.train()

for step in bar:
    
    optimizer_teacher.zero_grad()
    
    positive_sample, negative_sample, weight, mode = next(fb15k237)
    
    positive_sample = positive_sample.to(device)
    
    negative_sample = negative_sample.to(device)
    
    weight = weight.to(device)
    
    positive_score = teacher(sample=positive_sample)
    
    negative_score = teacher(sample=(positive_sample, negative_sample), mode=mode)
    
    loss_teacher = loss.Adversarial()(positive_score, negative_score, weight, alpha=0.5)
    
    loss_teacher.backward()
    
    optimizer_teacher.step()
    
    metric.update(loss_teacher.item())
    
    if step % 5 == 0:
        bar.set_description(f'Adversarial loss: {metric.get():6f}')
    
    if step % 2000 == 0:
        
        teacher = teacher.eval()
        
        score = evaluation(model=teacher, dataset=fb15k237.test_dataset(batch_size=8), device=device)
        
        teacher = teacher.train()
        
        print(score)
        
        # Set path HERE
        with open(f'./models/teacher_fb15k237_{score}.pickle', 'wb') as handle:
            
            pickle.dump(teacher, handle, protocol = pickle.HIGHEST_PROTOCOL)    

In [7]:
# Output of the previous training

teacher_name = 'teacher_fb15k237_HITS@10: 0.519716, HITS@1: 0.233778, HITS@3: 0.364971, MR: 192.433475, MRR: 0.329106.pickle'

with open(f'./models/{teacher_name}', 'rb') as handle:
    
    teacher = pickle.load(handle)    

In [ ]:
import numpy as np
import copy

def init_tensor(batch_size_entity, head, relation, tail):
    x = torch.zeros((1, batch_size_entity, 3))
    x[:,:,0] = head
    x[:,:,1] = relation
    x[:,:,2] = tail
    return x

In [ ]:
# Number of entities to consider to distill:
batch_size_entity = 20
batch_size_relation = 20

max_step = 40000

torch.manual_seed(42)
np.random.seed(42)

fb15k237 = datasets.FB15K237(
    batch_size=1000, 
    negative_sample_size=1, 
    seed=42, 
    shuffle=True
)

# Increasing the size of latents representations of the student allow to improve results.
student = model.RotatE(
    hidden_dim=1000, 
    n_entity=fb15k237.n_entity, 
    n_relation=fb15k237.n_relation, 
    gamma=0
)

student = student.to(device)

# Distillation process allow to handle different indexes between the student and the teacher.
# Distillation process allow to pre-compute batch dedicated to distillation.
distillation_process = distillation.Distillation(
    teacher_entities  = fb15k237.entities, 
    student_entities  = fb15k237.entities, 
    teacher_relations = fb15k237.relations, 
    student_relations = fb15k237.relations,
)

optimizer_student = torch.optim.Adam(
    filter(lambda p: p.requires_grad, student.parameters()), lr = 0.00005)

bar = tqdm.tqdm(range(1, max_step + 1), position=0)

# Creme online metric.
metric = stats.RollingMean(1000)

teacher = teacher.eval()
student = student.train()

for step in bar:
    
    positive_sample, negative_sample, weight, mode = next(fb15k237)
    
    batch_tensor_head     = []
    batch_tensor_relation = []
    batch_tensor_tail     = []

    entity_distribution = torch.tensor(
        np.random.randint(low=0, high=fb15k237.n_entity, size=batch_size_entity)
    ).view(1, batch_size_entity)
    
    relation_distribution = torch.tensor(
        np.random.randint(low=0, high=fb15k237.n_relation, size=batch_size_relation)
    ).view(1, batch_size_entity)

    optimizer_student.zero_grad()
    
    for head, relation, tail in positive_sample:
        
        head, relation, tail = head.item(), relation.item(), tail.item()
        
        head_distribution = copy.deepcopy(entity_distribution)
        head_distribution[0][0] = head
        
        tensor_head = init_tensor(
            head     = head_distribution, 
            relation = relation, 
            tail     = tail, 
            batch_size_entity = batch_size_entity
        )
        
        batch_tensor_head.append(tensor_head)
        
        tail_distribution = copy.deepcopy(entity_distribution)
        tail_distribution[0][0] = tail
        
        tensor_tail = init_tensor(
            head     = head, 
            relation = relation, 
            tail     = tail_distribution, 
            batch_size_entity = batch_size_entity
        )
        
        batch_tensor_tail.append(tensor_tail)
        
        relation_pre_distribution = copy.deepcopy(relation_distribution)
        relation_pre_distribution[0][0] = relation
        
        tensor_relation = init_tensor(
            head     = head, 
            relation = relation_pre_distribution, 
            tail     = tail, 
            batch_size_entity = batch_size_relation,
        )
        
        batch_tensor_relation.append(tensor_relation)
        
    # Create tensor [[(e_1, r_1, t_1),..(en, r_1, t_1)], ..,[(e1, r_3, t_2),..(e_n, r_3, t_2)]]
    # Tensor of size (batch_size, number of entity needed to compute the distribution probability, 3)
    teacher_head_tensor = torch.stack(batch_tensor_head).reshape(len(batch_tensor_head), batch_size_entity, 3).to(device=device, dtype=int)
    student_head_tensor = torch.stack(batch_tensor_head).reshape(len(batch_tensor_head), batch_size_entity, 3).to(device=device, dtype=int)
    
    teacher_relation_tensor = torch.stack(batch_tensor_relation).reshape(len(batch_tensor_relation), batch_size_entity, 3).to(device=device, dtype=int)
    student_relation_tensor = torch.stack(batch_tensor_relation).reshape(len(batch_tensor_relation), batch_size_entity, 3).to(device=device, dtype=int)
    
    teacher_tail_tensor = torch.stack(batch_tensor_tail).reshape(len(batch_tensor_tail), batch_size_entity, 3).to(device=device, dtype=int)
    student_tail_tensor = torch.stack(batch_tensor_tail).reshape(len(batch_tensor_tail), batch_size_entity, 3).to(device=device, dtype=int)
  
    # Distillation loss of heads
    loss_head = loss.KlDivergence()(
        teacher_score=teacher.distill(teacher_head_tensor), 
        student_score=student.distill(student_head_tensor)
    ) 
    
    # Distillation loss of relations.
    loss_relation = loss.KlDivergence()(
        teacher_score=teacher.distill(teacher_relation_tensor), 
        student_score=student.distill(student_relation_tensor)
    ) 
    
    # Distillation loss of tails.
    loss_tail = loss.KlDivergence()(
        teacher_score=teacher.distill(teacher_tail_tensor), 
        student_score=student.distill(student_tail_tensor)
    ) 
    
    # The loss of the student is equal to the sum of all losses.
    loss_student = loss_head + loss_relation + loss_tail
    
    metric.update(loss_student.item())
    
    loss_student.backward()

    optimizer_student.step()
    
    if step % 5 == 0:
    
        bar.set_description(f'Metric: {metric.get():6f}')
    
    if step % 1000 == 0:
        
        student = student.eval()
        
        score = evaluation.Evaluation()(model=student, dataset=fb15k237.test_dataset(batch_size=8), device=device)
        
        print(score)
        
        with open(f'./models/student_fb15k237_{score}.pickle', 'wb') as handle:
            
            pickle.dump(student, handle, protocol = pickle.HIGHEST_PROTOCOL)    
        
        student = student.train()

  0%|          | 0/40000 [00:00<?, ?it/s]/users/iris/rsourty/.local/lib/python3.6/site-packages/torch/nn/functional.py:1958: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
Metric: 0.171015:   2%|▏         | 999/40000 [06:45<4:25:28,  2.45it/s]

HITS@10: 0.332967, HITS@1: 0.129850, HITS@3: 0.215846, MR: 447.010554, MRR: 0.198185


Metric: 0.042698:   5%|▍         | 1999/40000 [17:10<4:16:00,  2.47it/s]  

HITS@10: 0.362577, HITS@1: 0.139817, HITS@3: 0.231872, MR: 318.034447, MRR: 0.214035


Metric: 0.024577:   7%|▋         | 2999/40000 [27:37<4:01:18,  2.56it/s]  

...

Metric: 0.001401:  62%|██████▏   | 24999/40000 [4:16:50<1:42:13,  2.45it/s]  

HITS@10: 0.508013, HITS@1: 0.221514, HITS@3: 0.351143, MR: 205.272256, MRR: 0.316201


Metric: 0.001395:  65%|██████▍   | 25999/40000 [4:27:16<1:32:11,  2.53it/s]  

HITS@10: 0.510554, HITS@1: 0.222931, HITS@3: 0.354417, MR: 202.126869, MRR: 0.317738


Metric: 0.001375:  67%|██████▋   | 26999/40000 [4:37:40<1:28:51,  2.44it/s]  